In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..')))
from src import *
from itertools import product

import numpy as np
from matplotlib import pyplot as plt

In [38]:
moments = np.array([-2, -1, 2, 3, 4])
ps = np.array([0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15])
system_sizes = np.array([100, 200, 400, 800, 1600, 3200])#, 6400]
realizations = 10

# push to larger sizes

def produce_data(ps, system_sizes, moments, realizations, bins = 50):
    widths = [L // bins for L in system_sizes]
    fractal_dimension = np.zeros((len(ps), len(system_sizes), len(moments)))
    params = product(enumerate(ps), enumerate(system_sizes), enumerate(moments))
    for (i, p), (j, L), (k, m) in params:
        dimension_data_reduced = 0
        for _ in range(realizations):
            ness_correlation = C_NESS(0, 0, A(L, p))
            _, eigenvectors = np.linalg.eig(ness_correlation)
            moment_data = calculate_moments(eigenvectors.T, widths[j], m)
            dimension_data = generalized_fractal_dimension(moment_data, widths[j], m, L)
            dimension_data_reduced += np.mean(list(dimension_data))
        fractal_dimension[i,j,k] = dimension_data_reduced / realizations
    return fractal_dimension

data = produce_data(ps, system_sizes, moments, realizations)
print(data)

[[[1.25459637 1.12132315 0.84040913 0.79193983 0.75665046]
  [1.14382455 1.07167639 0.88317561 0.84556226 0.81852764]
  [1.09681505 1.0499366  0.91458418 0.8843772  0.8582768 ]
  [1.07141133 1.03664387 0.93377161 0.90953676 0.88902669]
  [1.05664527 1.02951744 0.9476819  0.92724745 0.90996779]
  [1.04553108 1.02373484 0.9574448  0.94103244 0.92726764]]

 [[1.27606036 1.13073096 0.82456263 0.78012609 0.7458802 ]
  [1.1641801  1.08128469 0.8662248  0.827564   0.79854972]
  [1.11726503 1.06131878 0.89911819 0.86175693 0.83539832]
  [1.09404846 1.04896648 0.91510866 0.88549754 0.86267534]
  [1.07797164 1.04111924 0.92771091 0.90289591 0.88306595]
  [1.06775766 1.03556274 0.93828764 0.9157826  0.89766993]]

 [[1.29279715 1.14233433 0.80445665 0.75691587 0.72711639]
  [1.188394   1.09629001 0.84949156 0.80429784 0.76949555]
  [1.14519864 1.07700726 0.87498955 0.83661988 0.80924478]
  [1.12287526 1.06507034 0.8910314  0.85501643 0.8312869 ]
  [1.11084395 1.05902324 0.90110922 0.87011469 0.845

In [53]:
np.save("multifractal_data_tmp.npy", data)

In [55]:
%matplotlib qt

plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(nrows=5, figsize=(10, 12), )
for (k, q) in enumerate(moments):
    for (i, p) in enumerate(ps):
        ax[k].scatter(system_sizes, data[i, :, k], 
                    linewidth=2.0, label="p = {}".format(p))
        ax[k].plot(system_sizes, data[i, :, k], linewidth=0.5)
    ax[k].set_xlabel(r"$L$", labelpad=-15)
    ax[k].set_ylabel(r"$D_q$")
    ax[k].set_xscale('log')
    ax[k].set_title(r"q={}".format(q))
ax[2].legend(bbox_to_anchor=(1.0, 1.35))
plt.subplots_adjust(left = 0.1, right = 0.8, top = 0.97, bottom = 0.03, hspace = 0.5)
plt.show()

In [56]:
plt.savefig("rough_fractal_dimensions.png")